# 🛡️ Lab 8: Resilience Patterns
## Module 8 - Error Handling, Fallbacks, Circuit Breakers

**Duration:** 15 minutes

**Objectives:**
- Implement retry with exponential backoff
- Build model fallback chains
- Add confidence-based routing

In [ ]:
!pip install openai tenacity -q

In [ ]:
import os, json, time
from datetime import datetime
from enum import Enum

DEMO_MODE = False
client = None
MODEL_NAME = "gpt-4o"

try:
    from google.colab import userdata
    AZURE_OPENAI_KEY = userdata.get('AZURE_OPENAI_KEY')
    AZURE_OPENAI_ENDPOINT = userdata.get('AZURE_OPENAI_ENDPOINT')
    try: MODEL_NAME = userdata.get('AZURE_OPENAI_DEPLOYMENT')
    except: pass
    if AZURE_OPENAI_KEY and AZURE_OPENAI_ENDPOINT:
        if not AZURE_OPENAI_ENDPOINT.startswith('http'):
            AZURE_OPENAI_ENDPOINT = 'https://' + AZURE_OPENAI_ENDPOINT
        print(f"✅ Loaded. Model: {MODEL_NAME}")
    else: raise ValueError()
except: print("⚠️ DEMO MODE"); DEMO_MODE = True

if not DEMO_MODE:
    from openai import AzureOpenAI
    client = AzureOpenAI(api_key=AZURE_OPENAI_KEY, api_version="2024-06-01", azure_endpoint=AZURE_OPENAI_ENDPOINT)

## Part 1: Retry with Exponential Backoff

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential, retry_if_exception_type

@retry(stop=stop_after_attempt(3), wait=wait_exponential(multiplier=1, min=2, max=30))
def call_llm_with_retry(messages):
    if DEMO_MODE or not client:
        return "Demo response [DEMO]"
    response = client.chat.completions.create(model=MODEL_NAME, messages=messages)
    return response.choices[0].message.content

result = call_llm_with_retry([{"role": "user", "content": "What is 2+2?"}])
print(f"Result: {result}")

## Part 2: Model Fallback Chain

In [ ]:
class ModelFallbackChain:
    def __init__(self, client):
        self.client = client
        self.models = [MODEL_NAME, "gpt-4o-mini"] if not DEMO_MODE else []
        self.fallback = "I'm experiencing issues. Please try again or call 1-800-BANK."
    
    def invoke(self, messages):
        if DEMO_MODE or not self.client:
            return {"content": "Demo fallback response [DEMO]", "model_used": "demo", "fallback": True}
        
        for model in self.models:
            try:
                print(f"🔄 Trying {model}...")
                response = self.client.chat.completions.create(model=model, messages=messages)
                return {"content": response.choices[0].message.content, "model_used": model, "fallback": model != self.models[0]}
            except Exception as e:
                print(f"❌ {model} failed: {e}")
        
        return {"content": self.fallback, "model_used": "cached", "fallback": True}

chain = ModelFallbackChain(client)
result = chain.invoke([{"role": "user", "content": "What's my balance?"}])
print(f"\n✅ Response from: {result['model_used']}")
print(f"Content: {result['content'][:100]}...")

## Part 3: Circuit Breaker

In [ ]:
class CircuitBreaker:
    def __init__(self, threshold=3, timeout=60):
        self.threshold = threshold
        self.timeout = timeout
        self.failures = 0
        self.state = "CLOSED"
        self.last_failure = None
    
    def can_execute(self):
        if self.state == "CLOSED": return True
        if self.state == "OPEN" and self.last_failure:
            if (datetime.now() - self.last_failure).seconds >= self.timeout:
                self.state = "HALF_OPEN"
                return True
        return self.state == "HALF_OPEN"
    
    def record_success(self):
        if self.state == "HALF_OPEN": self.state = "CLOSED"; self.failures = 0
    
    def record_failure(self):
        self.failures += 1
        self.last_failure = datetime.now()
        if self.failures >= self.threshold: self.state = "OPEN"; print("🚨 Circuit OPEN")

cb = CircuitBreaker(threshold=3)
print("Simulating failures...")
for i in range(5):
    if cb.can_execute():
        print(f"  Request {i+1}: Executing (fail)")
        cb.record_failure()
    else:
        print(f"  Request {i+1}: BLOCKED")

print(f"\nState: {cb.state}, Failures: {cb.failures}")

## Part 4: Confidence Routing

In [ ]:
class ConfidenceAction(Enum):
    AUTO_EXECUTE = "auto"
    CONFIRM = "confirm"
    ESCALATE = "escalate"

def analyze_with_confidence(query):
    if DEMO_MODE or not client:
        # Demo confidence based on query type
        if "hours" in query.lower() or "balance" in query.lower():
            return {"response": "Branch hours: 9am-5pm [DEMO]", "confidence": 0.95, "action": ConfidenceAction.AUTO_EXECUTE}
        elif "invest" in query.lower() or "should i" in query.lower():
            return {"response": "Please consult advisor [DEMO]", "confidence": 0.4, "action": ConfidenceAction.ESCALATE}
        return {"response": "General response [DEMO]", "confidence": 0.75, "action": ConfidenceAction.CONFIRM}
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "system", "content": "Respond with JSON: {\"response\": \"...\", \"confidence\": 0.0-1.0}"},
                  {"role": "user", "content": query}],
        response_format={"type": "json_object"}
    )
    result = json.loads(response.choices[0].message.content)
    conf = result.get("confidence", 0.5)
    action = ConfidenceAction.AUTO_EXECUTE if conf >= 0.9 else (ConfidenceAction.CONFIRM if conf >= 0.7 else ConfidenceAction.ESCALATE)
    return {"response": result.get("response"), "confidence": conf, "action": action}

for q in ["What are your hours?", "Should I invest in crypto?"]:
    r = analyze_with_confidence(q)
    print(f"\nQuery: {q}")
    print(f"  Confidence: {r['confidence']:.0%} → {r['action'].value}")

## ✅ Lab 8 Complete!

**Key Takeaways:**
- Retry with backoff handles transient errors
- Fallback chains ensure availability
- Circuit breakers prevent cascading failures
- Confidence routing protects against uncertainty

**Next:** `09_cost_optimization.ipynb`